In [9]:
import pandas as pd
import numpy as np
from collections import Counter


In [10]:
data = pd.read_csv('small_train.tsv', sep='\t')
data.head()

,chest_pain,thalassemia,heart_disease
0,0,1,1
1,1,0,0
2,0,0,1
3,0,0,1
4,1,0,0


In [11]:
labels = data['heart_disease']
values, counts = np.unique(labels, return_counts=True)
values, counts

(array([0, 1]), array([14, 14]))

In [12]:
# get H(y)


In [13]:
# attribute 1

In [14]:
# get probability of each value in attribute 1
values, indices, counts = np.unique(data.iloc[:, 0], return_inverse = True, return_counts = True)
prob_neg = counts[0] / len(indices)
prob_pos = 1 - prob_neg
prob_neg, prob_pos, indices

(0.5714285714285714,
 0.4285714285714286,
 array([0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,
        0, 0, 1, 0, 1, 0]))

In [15]:
# get H(y|x)

# when x1 = 0
y_neg_x0 = []
y_pos_x0 = []
y_neg_x1 = []
y_pos_x1 = []

for idx in range(len(indices)):
    if indices[idx] == 0:
        y_neg_x0.append(labels[idx])
    else:
        y_pos_x0.append(labels[idx])

count_x0 = 0
for i in y_neg_x0:
    if i == 0:
        count_x0 += 1
    
prob_x0_y0 = count / len(y_neg_x0)
prob_x0_y1 = 1 - prob_x0_y0
entropy_y_x0 = -prob_x0_y0 * np.log2(prob_x0_y0) - prob_x0_y1 * np.log2(prob_x0_y1)

# when x1 = 1
for idx in range(len(indices)):
    if indices[idx] == 1:
        y_neg_x1.append(labels[idx])
    else:
        y_pos_x1.append(labels[idx])

count_x1 = 0
for i in y_neg_x1:
    if i == 0:
        count_x1 += 1
    
prob_x1_y0 = count_x1 / len(y_neg_x1)
prob_x1_y1 = 1 - prob_x1_y0
entropy_y_x1 = -prob_x1_y0 * np.log2(prob_x1_y0) - prob_x1_y1 * np.log2(prob_x1_y1)




NameError: name 'count' is not defined

In [ ]:
cond_entropy = prob_neg * entropy_y_x0 + prob_pos * entropy_y_x1
cond_entropy

0.7421685375402277

In [16]:
from decision_tree import Prep
from inspection import get_entropy

attr_num = data.shape[1] - 1
mi_ls = []
for x in range(attr_num):
    prob_neg, prob_pos, indices = Prep.calc_prob(data, x)
    cond_entropy = Prep.calc_conditional_entropy(indices, prob_neg, prob_pos, labels)
    entropy = get_entropy(labels)
    mi = Prep.calc_mi(entropy, cond_entropy)
    if mi > 0:
        mi_ls.append(mi)
    
max_mi = max(mi_ls)
# It is possible for different columns to have equal values for mutual information. 
# In this case, you should split on the first column to break ties
max_mi_attr_idx = mi_ls.index(max_mi)
max_mi_attr_idx
            


ImportError: cannot import name 'Prep' from 'decision_tree' (/Users/anyxling/Downloads/cmu_10601_hws/hw2/handout/decision_tree.py)

In [17]:
import numpy as np
import pandas as pd

class Node:
    def __init__(self):
        self.left = None
        self.right = None
        self.attr = None
        self.vote = None
        self.count = {'pos': 0, 'neg': 0}


class DecisionTree:
    def __init__(self, max_depth):
        self.root = Node()
        self.max_depth = max_depth

    def fit(self, data, labels):
        self._build_tree(self.root, data, labels, depth=0)

    def _build_tree(self, current_node, data, labels, depth):
        # Calculate positive and negative counts for current data
        pos_count = sum(labels)
        neg_count = len(labels) - pos_count

        current_node.count = {'pos': pos_count, 'neg': neg_count}

        # Base cases
        if depth == self.max_depth or len(data.columns) == 0:
            current_node.vote = 1 if pos_count >= neg_count else 0
            return
        
        best_attr_idx = self.best_split(data, labels)

        if best_attr_idx is None:
            current_node.vote = 1 if pos_count >= neg_count else 0
            return

        current_node.attr = best_attr_idx

        # Split data based on the best attribute
        left_mask = data.iloc[:, best_attr_idx] == 0
        right_mask = ~left_mask

        left_data = data[left_mask].drop(columns=data.columns[best_attr_idx])
        left_labels = labels[left_mask]

        right_data = data[right_mask].drop(columns=data.columns[best_attr_idx])
        right_labels = labels[right_mask]

        current_node.left = Node()
        current_node.right = Node()

        # Recursively build left and right subtree
        self._build_tree(current_node.left, left_data, left_labels, depth+1)
        self._build_tree(current_node.right, right_data, right_labels, depth+1)

    def best_split(self, data, labels):
        # Dummy function for the sake of example
        # Should implement mutual information calculation here
        return None  # Placeholder

    def print_tree(self, node=None, depth=0, attr_value=None):
        if node is None:
            node = self.root

        # Base case: if node is a leaf
        if node.vote is not None:
            print('| ' * depth + (attr_value + ": " if attr_value else "") + f"[{node.count['neg'] + node.count['pos']} {node.count['neg']}/{node.count['pos']}]")
            return
        
        print('| ' * depth + (attr_value + ": " if attr_value else "") + f"[{node.count['neg'] + node.count['pos']} {node.count['neg']}/{node.count['pos']}]")
        
        left_attr_value = f"{data.columns[node.attr]} = 0"
        self.print_tree(node.left, depth+1, left_attr_value)

        right_attr_value = f"{data.columns[node.attr]} = 1"
        self.print_tree(node.right, depth+1, right_attr_value)



In [18]:
# Example Usage
data = pd.DataFrame({
    'Feature1': [0, 1, 0, 1],
    'Feature2': [1, 1, 0, 0],
})
labels = pd.Series([0, 1, 1, 0])

tree = DecisionTree(max_depth=2)
tree.fit(data, labels)
tree.print_tree()

[4 2/2]


In [19]:
recitation_data = {
    'Outlook(X1)': ['sunny', 'overcast', 'rain', 'rain', 'sunny', 'sunny', 'rain', 'overcast'], 
    'Temperature(X2)':['hot', 'hot', 'mild', 'cool', 'mild', 'mild', 'mild', 'hot'], 
    'Humidity(X3)': ['high', 'high', 'high', 'normal', 'high', 'normal', 'normal', 'normal'], 
    'Play Tennis(Y)': [0, 1, 1, 1, 0, 1, 1, 1]
}

df = pd.DataFrame(recitation_data)
df

,Outlook(X1),Temperature(X2),Humidity(X3),Play Tennis(Y)
0,sunny,hot,high,0
1,overcast,hot,high,1
2,rain,mild,high,1
3,rain,cool,normal,1
4,sunny,mild,high,0
5,sunny,mild,normal,1
6,rain,mild,normal,1
7,overcast,hot,normal,1


In [20]:
# get_entropy correct!
from inspection import get_entropy
entropy = get_entropy(df.iloc[:, -1])
entropy

0.8112781244591328

In [21]:
from inspection import get_entropy
# get_cond_entropy
unique_val, counts = np.unique(df.iloc[:, 0], return_counts = True)
x1_overcast_mask = df.iloc[:, 0] == unique_val[0]
x1_overcast_df = df[x1_overcast_mask]
# val_overcast, counts_overcast = np.unique(x1_overcast_df.iloc[:, -1], return_counts = True)
print(x1_overcast_df.iloc[:, -1])
entropy_overcast = get_entropy(x1_overcast_df.iloc[:, -1])
entropy_overcast


1    1
7    1
Name: Play Tennis(Y), dtype: int64


/Users/anyxling/Downloads/cmu_10601_hws/hw2/handout/inspection.py:19: RuntimeWarning: divide by zero encountered in log2
  log_neg, log_pos = np.log2(prob_neg), np.log2(prob_pos)


0.0